In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re
import csv
from datetime import datetime
import os

# Creating folder and CSV file
os.makedirs('../data', exist_ok=True)

with open('../data/ebay_smartphones.csv', mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Title', 'Condition', 'Price', 'Rating', 'Items Sold', 'Scraped Time'])

    def extract_conditions(title):
        conditions = ['Excellent', 'Very Good', 'Fair', 'Good', 'Poor', 'Acceptable']
        for condition in conditions:
            if re.search(rf'\b{re.escape(condition)}\b', title, re.IGNORECASE):
                return condition
        return 'N/A'

    base_url = 'https://www.ebay.com/sch/i.html?_nkw=refurbished+smartphone&_ipg=50&_pgn={}'
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    }

    for page in range(1, 15):
        try:
            url = base_url.format(page)
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        items = soup.select('li.s-item')

        for item in items:
            # Title
            title_tag = item.select_one('div.s-item__title')
            title = title_tag.get_text(strip=True) if title_tag else 'N/A'

            # Condition
            condition = extract_conditions(title)

            # Price
            price_tag = item.select_one('span.s-item__price')
            price = price_tag.get_text(strip=True) if price_tag else 'N/A'

            # Rating
            rating_tag = item.select_one('div.x-star-rating span.clipped')
            rating = rating_tag.get_text(strip=True) if rating_tag else 'N/A'

            # Items Sold
            quantity_sold_tag = item.select_one('span.s-item__dynamic.s-item__quantitySold')
            items_sold = 'N/A'
            if quantity_sold_tag:
                sold_span = quantity_sold_tag.find('span', class_='BOLD')
                if sold_span:
                    items_sold = sold_span.get_text(strip=True)

            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            csv_writer.writerow([title, condition, price, rating, items_sold, timestamp])
            print(f"Page: {page}\nTitle: {title}\nCondition: {condition}\nPrice: {price}\nRating: {rating}\nItems Sold: {items_sold}\nTimeStamp: {timestamp}\n{'-'*60}")

        time.sleep(1)


PermissionError: [Errno 13] Permission denied: '../data/ebay_smartphones.csv'